In [49]:
import pandas as pd
from typing import List, Dict

df = pd.read_csv("restaurants_definitivo.csv", index_col = 0)
df

,Name,Address,Lat,Long,Number of Ratings,Restaurant Rating,Type of Restaurant,Number of Bookmarks,Price Range,Phone Number,Website,Instagram,Facebook,Timetable,url
id,,,,,,,,,,,,,,,
1,Freedom Cakes Diner,"Calle Cádiz 7,Madrid,Spain,28004",40.415876,-3.702805,470.0,4.5,['International'],766,Moderado,+34-910741347,http://www.freedomcakes.es/,http://www.instagram.com/freedomcakes,http://facebook.com/Freedomcakesvegan,"{'Monday': '08:00 - 00:00', 'Tuesday': '08:00 ...",https://www.happycow.net/reviews/freedom-cakes...
2,Mad Mad Vegan - Chueca,"Calle Pelayo 19,Madrid,Spain,28004",40.422698,-3.698609,410.0,5.0,"['American', 'Mexican']",612,Moderado,+34-914442077,https://madmadvegan.com/,http://www.instagram.com/madmadvegan,http://facebook.com/MADMADVEGAN,"{'Monday': '13:00 - 00:00', 'Tuesday': '13:00 ...",https://www.happycow.net/reviews/mad-mad-madri...
3,Frutas Prohibidas,"Calle del Conde Duque, 26,Madrid,Spain,28015",40.427467,-3.710349,241.0,5.0,"['International', 'Fusion']",555,Moderado,+34-911525533,http://www.frutasprohibidas.com/,http://www.instagram.com/frutasprohibidasbar,http://facebook.com/frutasprohibidasbar,"{'Monday': 'Closed', 'Tuesday': 'Closed', 'Wed...",https://www.happycow.net/reviews/frutas-prohib...
4,Mad Mad Vegan - Lavapiés,"Lavapiés 16,Madrid,Spain,28012",40.411113,-3.703039,216.0,4.5,"['American', 'Mexican']",465,Moderado,+34-910665299,https://madmadvegan.com/,http://www.instagram.com/madmadvegan,http://facebook.com/MADMADVEGAN,"{'Monday': '13:00 - 00:00', 'Tuesday': '13:00 ...",https://www.happycow.net/reviews/mad-mad-vegan...
5,Flower Burger,"Calle Luchana 13, Chamberí,Madrid,Spain,28010",40.430001,-3.701037,14.0,4.5,"['American', 'Western']",47,Barato,+34-910328968,https://www.flowerburger.it/,http://www.instagram.com/flowerburger_es,http://facebook.com/flowerburger.it,"{'Monday': '13:00 - 16:30, 19:30 - 23:30', 'Tu...",https://www.happycow.net/reviews/flower-burger...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,Morgano Crafted Beers,"Alamo 2, Boadilla del Monte, Madrid, Spain, 28660",40.405759,-3.876917,1.0,4.0,"['Middle Eastern', 'Spanish']",2,Barato,+34-695635423,No website,No instagram,http://facebook.com/MorganoCraftedBeers,"{'Monday': '19:00 - 01:00', 'Tuesday': '19:00 ...",https://www.happycow.net/reviews/morgano-craft...
745,Adagio Gelateria,"Avenida de Adolfo Suárez, 7, Boadilla Del Mont...",40.405413,-3.876999,1.0,4.0,['Italian'],0,No price range,+34-912859853,No website,http://www.instagram.com/adagiogelateria,http://facebook.com/adagiogelateria,"{'Monday': '10:00 - 22:30', 'Tuesday': '10:00 ...",https://www.happycow.net/reviews/adagio-gelate...
746,La Huella Vegana,"Calle de las Trece Rosas, 2,Rivas-vaciamadrid,...",40.376390,-3.540020,31.0,4.5,"['Western', 'Spanish']",42,Moderado,+34-917139958,http://www.lahuellavegana.com/,No instagram,http://facebook.com/lahuellavegana,"{'Monday': '08:30 - 23:00', 'Tuesday': '08:30 ...",https://www.happycow.net/reviews/la-huella-riv...


In [66]:
import pandas as pd
from typing import List, Dict

tipos_restaurantes: List[str] = [
    "Korean", "African", "American", "Asian", "Australian", "Brazilian", "British", 
    "Caribbean", "Chinese", "European", "French", "Fusion", "German", "Indian", 
    "International", "Italian", "Japanese", "Latin", "Mediterranean", "Mexican", 
    "Middle Eastern", "Spanish", "Taiwanese", "Thai", "Vietnamese", "Western"
]

total_tipos: Dict[str, Dict[str, float]] = {}

for tipo in tipos_restaurantes:
    filtro_tipo = df[df["Type of Restaurant"].str.contains(tipo, case=False, na=False)].copy()

    filtro_tipo.loc[filtro_tipo["Price Range"] == "Barato", "Price Range"] = 1
    filtro_tipo.loc[filtro_tipo["Price Range"] == "Moderado", "Price Range"] = 2
    filtro_tipo.loc[filtro_tipo["Price Range"] == "Caro", "Price Range"] = 3
    filtro_tipo.loc[~filtro_tipo["Price Range"].isin([1, 2, 3]), "Price Range"] = 0

    numero_tipo = len(filtro_tipo)
    numero_reviews = int(filtro_tipo["Number of Ratings"].sum())

    if numero_reviews > 0:
        distribucion = (filtro_tipo["Number of Ratings"] / numero_reviews) * filtro_tipo["Restaurant Rating"]
        media_ponderada = round(distribucion.sum(), 3)
    else:
        media_ponderada = 0

    valid_prices = filtro_tipo[filtro_tipo["Price Range"] != 0]["Price Range"]
    if len(valid_prices) > 0:
        media_precios = (round(valid_prices.sum() / len(valid_prices)))
    else:
        media_precios = 0

    total_tipos[tipo] = {
        "total": numero_tipo,
        "reviews": numero_reviews,
        "MP rating": media_ponderada,
        "Media precio": media_precios
    }

df_total_tipos = pd.DataFrame.from_dict(total_tipos)
transform = {
    1: "Barato",
    2: "Moderado",
    3: "Caro"
}

df_mod_media_precios = df_total_tipos.T[["Media precio"]].replace(transform)
df_mod_media_precios.T
df_transpose = df_total_tipos.T
df_transpose["Media Precios"] = df_mod_media_precios
df_def = df_transpose.drop(columns="Media precio")
df_def

,total,reviews,MP rating,Media Precios
Korean,4.0,10.0,3.400,Moderado
African,7.0,25.0,3.920,Moderado
American,85.0,1356.0,4.490,Moderado
Asian,86.0,412.0,3.823,Moderado
Australian,2.0,18.0,3.806,Moderado
Brazilian,8.0,35.0,4.571,Moderado
British,1.0,0.0,0.000,Moderado
Caribbean,3.0,5.0,3.200,Barato
Chinese,19.0,246.0,3.803,Barato
European,124.0,1241.0,4.164,Moderado


In [67]:
total_reviews_ordenados = df_def.sort_values("reviews", ascending=False)
total_reviews_ordenados[:10]

,total,reviews,MP rating,Media Precios
Spanish,227.0,4898.0,4.419,Moderado
International,101.0,2370.0,4.403,Moderado
Mediterranean,120.0,2351.0,4.397,Moderado
Western,80.0,1558.0,4.437,Moderado
Fusion,61.0,1519.0,4.431,Moderado
American,85.0,1356.0,4.490,Moderado
European,124.0,1241.0,4.164,Moderado
Italian,86.0,1078.0,4.565,Moderado
Mexican,37.0,1006.0,4.572,Moderado
Asian,86.0,412.0,3.823,Moderado


In [68]:
total_reviews_ordenados[-10:]

,total,reviews,MP rating,Media Precios
African,7.0,25.0,3.920,Moderado
Indian,7.0,22.0,3.818,Moderado
French,6.0,19.0,3.211,Barato
Australian,2.0,18.0,3.806,Moderado
Taiwanese,1.0,13.0,3.500,Barato
Korean,4.0,10.0,3.400,Moderado
Vietnamese,3.0,5.0,3.600,Moderado
Caribbean,3.0,5.0,3.200,Barato
German,2.0,1.0,4.000,Moderado
British,1.0,0.0,0.000,Moderado


In [69]:
total_tipos_ordenados = df_def.sort_values("total", ascending=False)
total_tipos_ordenados[:10]


,total,reviews,MP rating,Media Precios
Spanish,227.0,4898.0,4.419,Moderado
European,124.0,1241.0,4.164,Moderado
Mediterranean,120.0,2351.0,4.397,Moderado
International,101.0,2370.0,4.403,Moderado
Italian,86.0,1078.0,4.565,Moderado
Asian,86.0,412.0,3.823,Moderado
American,85.0,1356.0,4.490,Moderado
Western,80.0,1558.0,4.437,Moderado
Fusion,61.0,1519.0,4.431,Moderado
Japanese,54.0,271.0,3.873,Moderado


In [71]:
total_tipos_ordenados[-10:]


,total,reviews,MP rating,Media Precios
Indian,7.0,22.0,3.818,Moderado
French,6.0,19.0,3.211,Barato
Thai,5.0,26.0,3.500,Moderado
Korean,4.0,10.0,3.400,Moderado
Caribbean,3.0,5.0,3.200,Barato
Vietnamese,3.0,5.0,3.600,Moderado
German,2.0,1.0,4.000,Moderado
Australian,2.0,18.0,3.806,Moderado
British,1.0,0.0,0.000,Moderado
Taiwanese,1.0,13.0,3.500,Barato


In [72]:
total_rating_ordenados = df_def.sort_values("MP rating", ascending=False)
total_rating_ordenados[:10]

,total,reviews,MP rating,Media Precios
Middle Eastern,34.0,335.0,4.578,Moderado
Mexican,37.0,1006.0,4.572,Moderado
Brazilian,8.0,35.0,4.571,Moderado
Italian,86.0,1078.0,4.565,Moderado
American,85.0,1356.0,4.490,Moderado
Western,80.0,1558.0,4.437,Moderado
Fusion,61.0,1519.0,4.431,Moderado
Spanish,227.0,4898.0,4.419,Moderado
International,101.0,2370.0,4.403,Moderado
Mediterranean,120.0,2351.0,4.397,Moderado


In [73]:
total_rating_ordenados[-10:]

,total,reviews,MP rating,Media Precios
Indian,7.0,22.0,3.818,Moderado
Australian,2.0,18.0,3.806,Moderado
Chinese,19.0,246.0,3.803,Barato
Vietnamese,3.0,5.0,3.600,Moderado
Taiwanese,1.0,13.0,3.500,Barato
Thai,5.0,26.0,3.500,Moderado
Korean,4.0,10.0,3.400,Moderado
French,6.0,19.0,3.211,Barato
Caribbean,3.0,5.0,3.200,Barato
British,1.0,0.0,0.000,Moderado
